In [113]:
import gspread
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [114]:
service_account = gspread.service_account(filename='./config/expense-tracker-358105-199d116b0a6d.json')
workbook = service_account.open('badminton_tracking')
worksheet = workbook.get_worksheet(0)

In [115]:
df = pd.DataFrame(worksheet.get_all_records()).drop(["Timestamp", "result"], axis=1)

df.columns = ["date", "team_1_player_1", "team_1_player_2", "team_2_player_1", "team_2_player_2", "points_team_1", "points_team_2", "venue"]

df['winner'] = np.where(df.points_team_1 > df.points_team_2, 'team_1', 'team_2')
df['margin'] = abs(df.points_team_1 - df.points_team_2)
df['total_points_per_game'] = df["points_team_1"] + df["points_team_2"]

df = df.applymap(lambda x: f'{x}'.lower().strip() if isinstance(x, str) else x)

In [116]:
expense_data = pd.DataFrame(workbook.worksheet('expense_tracker').get_all_records()).set_index('date').reset_index()

In [117]:
expense_data

,date,amount,paid_by
0,2023-03-05,380,ajay
1,2023-03-06,200,raghotham
2,2023-03-07,200,ajay
3,2023-03-08,270,raghotham


In [125]:
def fn(x):
    return list(set(', '.join([', '.join(x[i]) for i in range(4)]).split(', ')))

players_on_date = df[['date', 'team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].copy()

players_on_date['date'] = players_on_date['date'].apply(lambda x: f'{datetime.strptime(x, "%m/%d/%Y").date()}')

players_on_date = players_on_date.groupby("date").agg({
    'team_1_player_1': 'unique',
    'team_1_player_2': 'unique',
    'team_2_player_1': 'unique',
    'team_2_player_2': 'unique'
})

players_on_date['players'] = players_on_date[['team_1_player_1', 'team_1_player_2', 'team_2_player_1', 'team_2_player_2']].apply(fn, axis=1)
players_on_date = players_on_date[['players']]
players_on_date['number_of_players'] = players_on_date['players'].str.len()

In [126]:
player_share = pd.merge(players_on_date.explode('players'), expense_data, how="inner", on="date")
player_share['share'] = player_share['amount'] / player_share['number_of_players']

In [127]:
player_share

,date,players,number_of_players,amount,paid_by,share
0,2023-03-05,ajay,5,380,ajay,76.0
1,2023-03-05,vinay,5,380,ajay,76.0
2,2023-03-05,raghotham,5,380,ajay,76.0
3,2023-03-05,srinidhi,5,380,ajay,76.0
4,2023-03-05,prateek,5,380,ajay,76.0
5,2023-03-06,ajay,4,200,raghotham,50.0
6,2023-03-06,vinay,4,200,raghotham,50.0
7,2023-03-06,raghotham,4,200,raghotham,50.0
8,2023-03-06,sameer,4,200,raghotham,50.0
9,2023-03-07,ajay,5,200,ajay,40.0


In [128]:
# balances = 
player_share.groupby('players').agg({
    'date': 'count',
    'share': 'sum'
})

,date,share
players,,
aakarsh,1,45.0
ajay,4,211.0
prateek,2,116.0
raghotham,4,211.0
sameer,3,135.0
srinidhi,2,121.0
vinay,4,211.0


In [132]:
balances = player_share.groupby(['players', 'paid_by'][::-1]).agg({
    'date': 'count',
    'share': 'sum'
}).reset_index()

balances = balances[balances['paid_by'] != balances['players']]

In [140]:
balances = pd.merge(
    balances,
    balances,
    left_on=['paid_by', 'players'],
    right_on=['players', 'paid_by'],
    how="left"
)
# balances

In [142]:
balances['owes'] = np.where(
    balances['paid_by_y'].isna(),
    balances['share_x'],
    np.where(
        balances['share_x'] > balances['share_y'],
        balances['share_x'] - balances['share_y'],
        0
    )
)

In [144]:
balances = balances[['players_x', 'paid_by_x', 'owes', 'date_x']]

balances.columns = ['player', 'owes_to', 'amount', 'for_days']

In [149]:
balances

,player,owes_to,amount,for_days
0,prateek,ajay,116.0,2
1,raghotham,ajay,21.0,2
2,sameer,ajay,40.0,1
3,srinidhi,ajay,76.0,1
4,vinay,ajay,116.0,2
5,aakarsh,raghotham,45.0,1
6,ajay,raghotham,0.0,2
7,sameer,raghotham,95.0,2
8,srinidhi,raghotham,45.0,1
9,vinay,raghotham,95.0,2
